### Проекционные методы решения краевой задачи для обыкновенного дифференциального уравнения второго порядка

Вариант 1

Дана граничная задача

$-(\frac{1}{2+x}u')' + cos(x)u = 1+x$

$u(-1) = u(1) = 0$

#### 1. Метод Ритца

Координатная система: $(1-x^2)P_i^{(1,1)}(x), \ i=0, 1, ...$

In [1]:
import basis_functions as bf
import numpy as np
import sympy
import pandas as pd

# возвращает набор коэффициентов c_i разложения U
# с использованием данных функций f, p, r
def Ritz_method(k, n):
    pols = bf.Jacobi(k, n)
    phis, dphis = bf.base_funs(k, n)

    A = np.zeros((n, n))
    b = np.zeros((n, 1))

    x = sympy.symbols('x')
    f = 1 + x
    for i in range(3):
        h = f * phis[i]
        b[i] = sympy.integrals.integrate(h, (x, -1, 1))

    # вспомогательные значения для интегрирования Гаусса
    x1 = 1 / 3 * np.sqrt(5 - 2 * np.sqrt(10 / 7))
    x2 = 1 / 3 * np.sqrt(5 + 2 * np.sqrt(10 / 7))
    c1 = (322 + 13 * np.sqrt(70)) / 900
    c2 = (322 - 13 * np.sqrt(70)) / 900
    x_i = [-x2, -x1, 0, x1, x2]
    c_i = [c2, c1, 128 / 225, c1, c2]

    # первый индекс - номер узла
    # второй - определеяет производная или просто значение (0 - значение, 1 - производная)
    # третий - номер функци Ф_i
    arr = [bf.base_funs_values(k, n, x_i[i]) for i in range(5)]

    def Gauss(nodes, coefs, i, j):
        s = 0
        for k in range(len(nodes)):
            tmp_1 = ((1 / (2 + nodes[k])) * arr[k][1][j] * arr[k][1][i] + np.cos(nodes[k]) * arr[k][0][i] * arr[k][0][j])
            s += coefs[k] * tmp_1
        return s


    for i in range(n):
        for j in range(n):
            A[i][j] = Gauss(x_i, c_i, i, j)

    coeffs = np.linalg.solve(A, b)
    return coeffs, A
value = []
dot1, dot2, dot3 = -0.5, 0.0, 0.5

# значение полученное методом Рунге-Кутта с шагом 0.1
exact_value = [0.270198, 0.529251, 0.632899]

# вычисляем значения при n от 3 до 7
for n in range(3, 8):
    coeffs, A = Ritz_method(1, n)
    res = [0.0] * 3
    
    phi_dot1 = bf.base_funs_values(1, n, dot1)[0]
    phi_dot2 = bf.base_funs_values(1, n, dot2)[0]
    phi_dot3 = bf.base_funs_values(1, n, dot3)[0]

    for i in range(3):
        res[0] += coeffs[i] * phi_dot1[i]
        res[1] += coeffs[i] * phi_dot2[i]
        res[2] += coeffs[i] * phi_dot3[i]

    errs = [exact_value[k] - res[k] for k in range(3)]
    arr = [np.linalg.cond(A, np.inf),
           np.round(res[0], 5),
           np.round(res[1], 5),
           np.round(res[2], 5),
           np.round(errs[0], 5),
           np.round(errs[1], 5),
           np.round(errs[2], 5)]
    'arr = [res[0], res[1], res[2], errs[0], errs[1], errs[2]]'
    value.append(arr)


column = [
    "$\mu_{\infty}A$",
    "y(-0.5)",
    "y(0)",
    "y(0.5)",
    "y* - y _(-0.5)",
    "y* - y _(0.0)",
    "y* - y _(0.5)"
]
indexes = [3, 4, 5, 6, 7]
table = pd.DataFrame(data = value, columns=column, index=indexes)
table.columns.name = "n"

table

n,$\mu_{\infty}A$,y(-0.5),y(0),y(0.5),y* - y _(-0.5),y* - y _(0.0),y* - y _(0.5)
3,3.784387,[0.26742],[0.54346],[0.61822],[0.00278],[-0.0142],[0.01468]
4,4.930445,[0.26784],[0.54148],[0.61948],[0.00236],[-0.01223],[0.01342]
5,20.627862,[0.26799],[0.54219],[0.61933],[0.00221],[-0.01294],[0.01357]
6,117.849356,[0.26066],[0.54475],[0.63201],[0.00954],[-0.0155],[0.00089]
7,293.956035,[0.37366],[0.25301],[0.75022],[-0.10346],[0.27624],[-0.11733]
